In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn.metrics

import scipy.linalg
import scipy.spatial.distance
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('ytick', labelsize=7)
import seaborn as sns
import os
import psutil
import polars as pl
import cuml
import pandas as pd
from typing import List
from sklearn.preprocessing import StandardScaler
import numpy as np 
from collections import OrderedDict

In [17]:
featdir = "outputs/results/"
PROJECT_ROOT = "/share/data/analyses/benjamin/Single_cell_project/DP_BEACTICA/"
REG_PARAM = 1e-2

In [18]:
def find_file_with_string(directory, string):
    """
    Finds a file in the specified directory that contains the given string in its name.

    Args:
    directory (str): The directory to search in.
    string (str): The string to look for in the file names.

    Returns:
    str: The path to the first file found that contains the string. None if no such file is found.
    """
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"The directory {directory} does not exist.")
        return None

    # Iterate through all files in the directory
    for file in os.listdir(directory):
        if string in file:
            return os.path.join(directory, file)

    # Return None if no file is found
    return None


## Load metadata and features

In [8]:
meta = pd.read_csv(os.path.join(PROJECT_ROOT, "inputs", "metadata", "metadata_deepprofiler_beactica.csv")).drop_duplicates(inplace = False)
meta = meta.sort_values(by=['Metadata_Well', 'Metadata_Site'])
meta['Metadata_cmpdName'] = meta['Metadata_cmpdName'].str.upper()
meta["Metadata_cmpdNameConc"] = meta["Metadata_cmpdName"] +   " " + meta["Metadata_cmpdConc"].astype(str)
meta_pl = pl.DataFrame(meta).drop('Unnamed: 0.1', 'Unnamed: 0', "AR", "ER", "RNA", "AGP", "DNA", "Mito")
meta_pl = meta_pl.unique()

validation = meta_pl

In [23]:
import tqdm
import polars as pl
plates = validation["Metadata_Plate"].unique()
feat_out = "outputs/results/parquets"
master_df = pl.DataFrame()
for p in tqdm.tqdm(plates):
    feature_df = pl.read_parquet(find_file_with_string(os.path.join(PROJECT_ROOT, featdir), p))
    feature_df = feature_df.join(
    validation,
    on=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'],  # columns to join on
    how='inner')
    master_df = pl.concat([master_df, feature_df])

  0%|          | 0/8 [00:00<?, ?it/s]

/share/data/analyses/benjamin/Single_cell_project/DP_BEACTICA/outputs/results/features/PB000052


IsADirectoryError: expected a file path; '/share/data/analyses/benjamin/Single_cell_project/DP_BEACTICA/outputs/results/features/PB000052' is a directory

### Multiple MoA analysis

In [ ]:
duplicates = (meta_pl.groupby(['Site', 'Plate', 'Well', 'Metadata_cmpdName'])
               .agg(pl.count())
               .filter(pl.col('count') > 1))

print(duplicates)

In [ ]:
features = [col for col in master_df.columns if "Feature" in col]
meta_features = [ col for col in master_df.columns if col not in features]

### Normalization

In [ ]:
import scipy.stats
from sklearn.preprocessing import StandardScaler, RobustScaler
import pycytominer as pm

def prep_data(df1, features, meta_features, plate):
    mask = ~df1['Metadata_cmpdName'].isin(['BLANK', 'UNTREATED', 'null'])
    filtered_df = df1[mask]
    temp1 = filtered_df.copy()
    temp1 = temp1.loc[(temp1['Metadata_Plate'] == plate)]
    data_norm = pm.normalize(profiles = temp1, features =  features, meta_features = meta_features, samples = "Metadata_cmpdName == '[DMSO]'", method = "mad_robustize")
    print("Feature selection starts, shape:", data_norm.shape)
    df_selected = pm.feature_select(data_norm, features = features, operation = ['correlation_threshold', 'drop_na_columns'], corr_threshold=0.8)
    print('Number of columns removed:', data_norm.shape[1] - df_selected.shape[1])
    removed_cols = set(data_norm.columns) - set(df_selected.columns)
    #out = df_selected.dropna().reset_index(drop = True)
    #print('Number of NA rows removed:', df_selected.shape[0] - out.shape[0])
    df_selected["Metadata_cmpdNameConc"] = df_selected["Metadata_cmpdName"] + df_selected["Metadata_cmpdConc"].astype(str)
    return data_norm, removed_cols

In [ ]:
comp = list(master_df["Metadata_cmpdName"].unique())
mad_norm_df = pl.DataFrame()
drop_cols = {}
for p in tqdm.tqdm(plates):
    filtered_df = master_df.filter(pl.col('Metadata_Plate') == p)
    temp_pandas = filtered_df.to_pandas()
    temp_processed, dropped_cols = prep_data(temp_pandas, features, meta_features, p)
    drop_cols[p] = list(dropped_cols)
    temp_polars = pl.from_pandas(temp_processed)
    mad_norm_df = pl.concat([mad_norm_df, temp_polars])

mad_norm_df = mad_norm_df.filter(pl.col("Metadata_cmpdName").is_not_null())

In [ ]:
cols_to_drop = list(set().union(*drop_cols.values()))
mad_norm_df = mad_norm_df.drop(cols_to_drop)
features_fixed = [col for col in mad_norm_df.columns if col not in meta_features]

In [ ]:
mad_norm_df.write_parquet('/home/jovyan/share/data/analyses/benjamin/Single_cell_project_rapids/sc_profiles_normalized_BEACTICA.parquet')